### 설문 조사 데이터를 오버샘플링 하기 위한 코드
- 샘플링 수를 늘려보자 !

In [10]:
import pandas as pd
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
# 랜덤 오버 샘플링을 위한 라이브러리
from imblearn.over_sampling import RandomOverSampler

#### 원본 데이터 단순 복제로 데이터 수를 10만개로 늘리기

In [21]:
# 데이터 로드
data = pd.read_csv('./survey.csv')

# 원하는 샘플 수
target_samples = 100000
replication_factor = target_samples // len(data)  # 필요한 복제 횟수 계산
additional_samples = target_samples % len(data)   # 추가로 필요한 샘플 수

# 데이터 복제
oversampled_data = pd.concat([data] * replication_factor + [data.iloc[:additional_samples]])

# 결과 저장
oversampled_data.to_csv('./survey_replicated.csv', index=False)

print(f"데이터 복제 완료: 총 샘플 수 {len(oversampled_data)}")


데이터 복제 완료: 총 샘플 수 100000


#### 랜덤 오버 샘플링을 통해 전체 데이터의 수를 늘리기

- 단, 랜덤 오버 샘플링은 이산형 데이터에 사용이 가능
- 따라서 '0', '0.25', '0.5', '0.75', '1'을 범주화하여 제한을 우회
- 이후 다시 범주형 데이터를 수치로 롤백하여 원본 데이터의 형식을 유지
- 이 외의 숫자가 혹시 포함되어 있다면 디버깅을 위해 -1로 설정

- 랜덤 오버 샘플링을 위해 각 열을 독립적으로 오버 샘플링 후 다시 합쳐주는 방식 선택

In [22]:
# 데이터 로드 및 함수 정의
def discretize(value):
    if value == 0:
        return 'Very Low'
    elif value == 0.25:
        return 'Low'
    elif value == 0.5:
        return 'Medium'
    elif value == 0.75:
        return 'High'
    elif value == 1:
        return 'Very High'
    return 'Unknown'

def continuousize(category):
    mapping = {'Very Low': 0, 'Low': 0.25, 'Medium': 0.5, 'High': 0.75, 'Very High': 1}
    return mapping.get(category, -1)

original_data = pd.read_csv('./survey_replicated.csv')
categories = ['work', 'edu', 'free_time', 'health', 'chores']

# 범주화 및 원-핫 인코딩
X_encoded = pd.get_dummies(original_data.drop(categories, axis=1))
oversampled_dataframes = []
ros = RandomOverSampler(random_state=42)

# 각 범주에 대해 오버샘플링 적용
for category in categories:
    y = original_data[category].apply(discretize)
    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)
    y_resampled = y_resampled.apply(continuousize)
    df_resampled = pd.DataFrame(X_resampled, columns=X_encoded.columns)
    df_resampled[category] = y_resampled
    oversampled_dataframes.append(df_resampled)

# 모든 결과를 하나의 DataFrame으로 병합
final_df = oversampled_dataframes[0]
for df in oversampled_dataframes[1:]:
    final_df = final_df.merge(df, on=list(X_encoded.columns), how='inner')

# 반복적 오버샘플링으로 추가 증가
for _ in range(200):  # 반복 횟수에 따라 조정 가능
    final_df, _ = ros.fit_resample(final_df, final_df.columns)

# 결과 저장
final_df.to_csv('./survey_random_oversampled.csv', index=False)
print(f"오버샘플링 데이터 저장 완료: survey_random_oversampled.csv, 샘플 수: {len(final_df)}")


MemoryError: Unable to allocate 5.42 GiB for an array with shape (1, 726969060) and data type int64

#### 랜덤 오버 샘플링을 통해 늘어난 데이터 수를 바탕으로 smote 적용

In [27]:
# 데이터 로드
original_data = pd.read_csv('./survey_replicated.csv')

# 데이터 스케일링 및 클러스터링
scaler = StandardScaler()
y_scaled = scaler.fit_transform(original_data[['work', 'edu', 'free_time', 'health', 'chores']])

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(y_scaled)

# 범주형 데이터 원-핫 인코딩 및 특성 분리
X_encoded = pd.get_dummies(original_data.drop(['work', 'edu', 'free_time', 'health', 'chores'], axis=1))
X = pd.concat([X_encoded, original_data[['age', 'gender']]], axis=1)

# SMOTE 적용 (연속형 데이터 포함)
# n_neighbors 값 조정: 가장 작은 클러스터의 크기보다 작게 설정
min_cluster_size = min(pd.Series(clusters).value_counts())
n_neighbors = max(min(2, min_cluster_size - 1), 1)  # 최소 1, 최대 (가장 작은 클러스터 크기 - 1) 사이
smote = SMOTE(random_state=42, k_neighbors=n_neighbors)
X_resampled, clusters_resampled = smote.fit_resample(X, clusters)

# 연속형 데이터 복원: 각 클러스터의 중심을 사용하여 연속형 데이터를 복원
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
oversampled_y = [cluster_centers[cluster] for cluster in clusters_resampled]

# 오버샘플링된 데이터 프레임 생성
oversampled_df = pd.DataFrame(X_resampled, columns=X.columns)
oversampled_df[['work', 'edu', 'free_time', 'health', 'chores']] = pd.DataFrame(oversampled_y, columns=['work', 'edu', 'free_time', 'health', 'chores'])

# 결과 저장 및 출력
oversampled_df.to_csv('./survey_oversampled.csv', index=False)
print("오버샘플링 데이터 저장 완료: survey_oversampled.csv")


오버샘플링 데이터 저장 완료: survey_oversampled.csv


#### 오버 샘플링 된 데이터들을 정제하는 과정

In [28]:
# 오버샘플링된 데이터 로드
oversampled_data = pd.read_csv('./survey_oversampled.csv')

# 'job' 관련 열 추출 및 변환
job_columns = [col for col in oversampled_data.columns if col.startswith('job_')]
oversampled_data['job'] = oversampled_data[job_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 'mbti' 관련 열 추출 및 변환
mbti_columns = [col for col in oversampled_data.columns if col.startswith('mbti_')]
oversampled_data['mbti'] = oversampled_data[mbti_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 불필요한 원-핫 인코딩 열 제거
oversampled_data.drop(columns=job_columns + mbti_columns, inplace=True)

# 결과 저장
oversampled_data.to_csv('./survey_oversampled_fixed.csv', index=False)

print("데이터 업데이트 완료: survey_oversampled_fixed.csv")


데이터 업데이트 완료: survey_oversampled_fixed.csv


In [30]:
# 마지막으로 value 수정
fix_data = pd.read_csv('./survey_oversampled_fixed.csv')

# 불필요한 column들 제거
fix_data.drop(columns=['age.1','gender.1'], inplace=True)

# 'job' cloumn의 'job' 값을 'job_seeker'로 수정
fix_data['job'] = fix_data['job'].map(
    lambda x: 'job_seeker' if x == 'job' else x
    )

# 열 순서 재배열
columns_ordered = ['age', 'gender', 'job', 'mbti', 'edu', 'free_time', 'health', 'chores']
fix_data = fix_data[columns_ordered]

fix_data.to_csv('./survey_data.csv')

In [34]:
fix_data.describe()

,age,gender,edu,free_time,health,chores
count,115380.000000,115380.000000,115380.000000,115380.000000,115380.000000,115380.000000
mean,25.707748,0.458390,0.712497,0.777084,0.789576,0.506252
std,3.871719,0.498268,0.094515,0.166641,0.206694,0.135302
min,12.000000,0.000000,0.593741,0.562502,0.499976,0.343757
25%,23.000000,0.000000,0.593741,0.562502,0.499976,0.343757
50%,27.000000,0.000000,0.718751,0.800000,0.900000,0.500000
75%,28.000000,1.000000,0.825000,0.968751,0.968751,0.675000
max,34.000000,1.000000,0.825000,0.968751,0.968751,0.675000


----------------------------------------------------

### 데이터를 DB에 저장하기 위해 정제

In [9]:
# 파일 경로
path = './survey_oversampled_fixed.csv'

# CSV 파일 읽기
data = pd.read_csv(path)

# 나이 범위를 age_range 컬럼으로 추가
bins = [0, 20, 25, 30, 100]
labels = ['0-19', '20-24', '25-29', '30-']
data['age_range'] = pd.cut(data['age'], bins=bins, labels=labels, right=False)

# age_range 행을 기준으로 분류하여 각 변수의 가중치 평균 계산
grouped_means = data.groupby('age_range')[['work', 'edu', 'free_time', 'health', 'chores']].mean()

# 결과 출력
grouped_means

C:\Users\SeonB2\AppData\Local\Temp\ipykernel_30612\1794354282.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_means = data.groupby('age_range')[['work', 'edu', 'free_time', 'health', 'chores']].mean()


,work,edu,free_time,health,chores
age_range,,,,,
0-19,0.882353,0.720588,0.867647,0.838235,0.544118
20-24,0.752941,0.688235,0.597059,0.685294,0.517647
25-29,0.598760,0.764273,0.827710,0.859141,0.470012
30-,0.882353,0.720588,0.867647,0.838235,0.544118
